In [ ]:
!pip install syllables
import json, requests, syllables, wordcloud, textblob, nltk, matplotlib.pyplot as plt
nltk.download("punkt", quiet = True)

twitter_handles = ["@sprintcare", "@Ask_Spectrum", "@AskPlayStation", "@XboxSupport", "@UPSHelp", "@AmazonHelp", "@hulu_support", "@comcastcares", "@TMobileHelp", "@Uber_Support", "@SpotifyCares", "@AppleSupport"]

response = requests.get("https://dgoldberg.sdsu.edu/515/customer_service_tweets_full.json")

if response:
    data = json.loads(response.text)

#calculate flesch-kincaid grade level for thr fkgl analysis if chosen by the user (seperated for clarity)
    def calculate_fkgl(text):
        words = text.split()
        total_words = len(words)
        total_sentences = text.count('.') + text.count('!') + text.count('?')
        total_syllables = sum(syllables.estimate(word) for word in words)
        if total_sentences == 0:
            return 0
        fkgl = 0.39 * (total_words / total_sentences) + 11.8 * (total_syllables / total_words) - 15.59
        return fkgl

#calculate analysis for each analysis type chosen by the user (seperated analysis section for clarity)
    def perform_analysis(handle, analysis_type):
        handle_tweets = [tweet['Text'] for tweet in data if handle.lower() == tweet['Company'].lower()]
        if not handle_tweets:
            print(f"No tweets found for {handle}.")
            return
        if analysis_type == 'polarity':
            sentiment_scores = [textblob.TextBlob(tweet).sentiment.polarity for tweet in handle_tweets]
            average_score = sum(sentiment_scores) / len(sentiment_scores)
            print(f"{handle}: {average_score}")
        elif analysis_type == 'subjectivity':
            sentiment_scores = [textblob.TextBlob(tweet).sentiment.subjectivity for tweet in handle_tweets]
            average_score = sum(sentiment_scores) / len(sentiment_scores)
            print(f"{handle}: {average_score}")
        elif analysis_type == 'fkgl':
            fkgl_scores = [calculate_fkgl(tweet) for tweet in handle_tweets]
            average_score = sum(fkgl_scores) / len(fkgl_scores)
            print(f"{handle}: {average_score}")
        elif analysis_type == 'wordcloud':
            all_tweets = " ".join(handle_tweets)
            cloud = wordcloud.WordCloud(width=800, height=400).generate(str(all_tweets))
            plt.figure(figsize=(8, 4))
            plt.imshow(cloud, interpolation='bilinear')
            plt.axis("off")
            plt.show()
        else:
            print("Sorry, connection error.")


#start the loop that asks what analysis and what twitter handle the user would like
    print("Welcome to the customer service analyzer!")
    while True:
        analysis_type = input("Which analysis would you like to perform (polarity/subjectivity/FKGL/wordcloud)? ").lower()
        handle = input("Which Twitter handle would you like to analyze? ").lower()
        if handle not in [i.lower() for i in twitter_handles]:
            print("Unsupported twitter handle.")
            continue

        perform_analysis(handle, analysis_type)

        another_analysis = input("Would you like to run another analysis (yes/no)? ").lower()
        if another_analysis != 'yes':
            break